In [1]:
import numpy as np

import sensors
import em_stoch

In [2]:
DIST_RATIO = 0.5

In [3]:
Num_sensors1 = 5
Num_emitters1 = 1
sample_size1 = 100
theta1_rad = [0.7] # Угловые координаты источников (DoA) в радианах
theta1_deg = np.rad2deg(theta1_rad) # Угловые координаты источников (DoA) в градусах
P_1 = 1 * np.eye(Num_emitters1, dtype=np.float64) # Ковариация сигналов
Q_1 = 1.1 * np.eye(Num_sensors1, dtype=np.float64) # Ковариация шумов
A1 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors1).reshape(-1,1) * 
             np.sin(theta1_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S1 = sensors.gss(Num_emitters1, sample_size1, P_1)
N1 = sensors.gss(Num_sensors1, sample_size1, Q_1)
X1 = (A1 @ S1.T + N1.T).T

Num_sensors2 = 5
Num_emitters2 = 2
sample_size2 = 100
theta2_rad = np.array([-0.5, 0.7], dtype=np.float64) # Угловые координаты источников (DoA) в радианах
theta2_deg = np.rad2deg(theta2_rad) # Угловые координаты источников (DoA) в градусах
P_2 = 1 * np.eye(Num_emitters2, dtype=np.float64) # Ковариация сигналов
Q_2 = 1.1 * np.eye(Num_sensors2, dtype=np.float64) # Ковариация шумов
A2 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors2).reshape(-1,1) * 
             np.sin(theta2_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S2 = sensors.gss(Num_emitters2, sample_size2, P_2)
N2 = sensors.gss(Num_sensors2, sample_size2, Q_2)
X2 = (A2 @ S2.T + N2.T).T

In [4]:
X1_with_mv = sensors.MCAR(X1, [2,4], [0.5, 0.5])
X2_with_mv = sensors.MCAR(X2, [2,4], [0.5, 0.5])

In [5]:
X1_with_mv

array([[ 1.23286261e+00+0.9418559j , -1.85659807e-02-2.84696988j,
                    nan+0.j        ,  2.32727443e-01-0.74680402j,
                    nan+0.j        ],
       [-1.84555688e+00-0.03615284j,  5.79029362e-01+0.98613426j,
         2.52068362e-01-0.41735757j, -8.97514167e-01+0.04994758j,
                    nan+0.j        ],
       [-1.09664728e+00+1.1646421j , -1.72871621e-01-0.7934921j ,
         1.76743201e+00-1.60581369j, -1.22187416e+00+0.6652129j ,
                    nan+0.j        ],
       [-3.19705011e+00-1.03130086j,  6.19777131e-01+1.91974302j,
         2.07216841e+00-3.48689659j, -2.50062242e+00-0.98218031j,
        -3.00018978e-01+1.93779552j],
       [-1.05663395e+00-0.81800175j, -7.86290658e-01+1.43634167j,
                    nan+0.j        , -3.21607249e-01-0.98971863j,
         3.63068447e-01+1.76799945j],
       [-4.64436551e-01+1.06533348j,  4.39830960e-01+1.36970282j,
                    nan+0.j        , -1.05030240e+00+0.27738488j,
                  

In [6]:
print(np.trace(sensors.initial_Cov(X1_with_mv)))
print(np.trace(sensors.complex_cov(X1)))

(11.017603785443473+0j)
(11.105494677649398+0j)


In [7]:
EM_theta1_rad, EM_P1, lhd_1 = em_stoch.multi_start_EM(X1, Num_emitters1, Q_1, 10, 
                                                         max_iter=30, 
                                                         rtol=1e-6)

0-th start
theta=[0.15335215],P=[0.12062233-1.20247748e-17j]
res=[[0.12062227-1.02900513e-17j]]
new_angles=[0.15335214]
1-th start
theta=[0.13636065],P=[0.14490204-3.87895961e-17j]
res=[[0.14490196-4.22590431e-17j]]
new_angles=[0.13636065]
2-th start
theta=[1.40627821],P=[0.09965251+8.53371115e-18j]
res=[[0.09965247+1.02684346e-17j]]
new_angles=[1.40627821]
3-th start
theta=[-0.15355298],P=[0.00044721+0.j]
res=[[0.00044721+5.29395592e-23j]]
new_angles=[-0.15355298]
4-th start
theta=[0.5300775],P=[1.72866676+1.11714037e-16j]
res=[[1.72866208-2.76864022e-16j]]
new_angles=[0.53007749]
likelihood is -874.9333755055419 on iteration 0
res=[[1.72865741-5.55111512e-16j]]
new_angles=[0.57340785]
likelihood is -813.69393700385 on iteration 1
res=[[1.72865466-4.4408921e-16j]]
new_angles=[0.60578729]
likelihood is -778.0680105080271 on iteration 2
res=[[1.72865301-2.49800181e-16j]]
new_angles=[0.62962093]
likelihood is -758.603303259131 on iteration 3
res=[[1.72865197+2.77555756e-16j]]
new_angles=

In [8]:
print(EM_theta1_rad, EM_P1)

[0.69245571] [[1.72864139]]


In [9]:
EM_theta1_deg = np.rad2deg(EM_theta1_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(EM_theta1_deg-theta1_deg)},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta1_rad)-np.sin(theta1_rad))}.")

Погрешность в смысле разности углов в градусах:       [0.43225575],
Погрешность в смысле разности синусов углов:       [0.00578847].


In [10]:
EM_theta2_rad, EM_P2, lhd_2 = em_stoch.multi_start_EM(X2_with_mv, Num_emitters2, Q_2, 10, 
                                                         max_iter=30, 
                                                         rtol=1e-3)

0-th start
theta=[-1.41744418  0.15335215],P=[0.00044721+0.00000000e+00j 0.12665156+2.43107915e-17j]
res=[[0.0004468 +6.74801309e-23j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.10671344+2.36956326e-17j]]


new_angles=[-1.56984761  0.1533176 ]
likelihood is -1135.8713087133895 on iteration 0
res=[[0.00044667+7.85601816e-24j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.10283953+9.35439134e-20j]]
new_angles=[-1.57079633  0.1429595 ]
likelihood is -1135.3586853360441 on iteration 1
res=[[0.00044655+5.41028087e-24j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.09939859-2.37109473e-19j]]
new_angles=[-1.57079633  0.13469842]
likelihood is -1135.1057765022847 on iteration 2
res=[[0.00044644-6.03532734e-24j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.09642925+1.14009346e-18j]]
new_angles=[-1.57079633  0.12800639]
likelihood is -1135.008172913024 on iteration 3
res=[[0.00044633-4.59591448e-24j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.09386286+2.40443458e-19j]]
new_angles=[-1.57079633  0.12250411]
likelihood is -1135.0057373541226 on iteration 4
res=[[0.00044624-1.91574242e-23j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.091

In [11]:
print(EM_theta2_rad, EM_P2)

[-0.57849607  0.6792158 ] [[4.45605631e-04 0.00000000e+00]
 [0.00000000e+00 8.46508676e-01]]


In [12]:
EM_theta2_deg = np.rad2deg(EM_theta2_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(np.sort(EM_theta2_deg)-np.sort(theta2_deg))},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta2_rad)-np.sin(theta2_rad))}.")

Погрешность в смысле разности углов в градусах:       [4.49749338 1.19084674],
Погрешность в смысле разности синусов углов:       [0.0673398  0.01603463].


In [13]:
em_stoch.incomplete_lkhd(X1_with_mv, theta1_rad, P_1, Q_1)

np.float64(-600.3004356335736)

In [14]:
em_stoch.incomplete_lkhd(X1_with_mv, EM_theta1_rad, EM_P1, Q_1)

np.float64(-605.7015188008291)

In [15]:
em_stoch.incomplete_lkhd(X2_with_mv, theta2_rad, P_2, Q_2)

np.float64(-734.0331279090324)

In [16]:
em_stoch.incomplete_lkhd(X2_with_mv, EM_theta2_rad, EM_P2, Q_2)

np.float64(-954.6452352549162)